In [339]:
pip install lxml

In [340]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import lxml
import re

## Meeting Plan for Nicolas Ouellet and Benedicte Knudson:
We colaborated over Zoom and in person **three** times and used a shared Github Repository that can be found [here](https://github.com/NickOuellet/Fly-Me-To-The-Moon). <br />
We met: <br />
10/11: 5-7pm <br />
10/20: 10-12pm <br />
10/21: 8-11pm <br />


## Step 1 - Scrape SpaceWeatherLive.com

In [341]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
r = requests.get("https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares.html",timeout=10,headers=headers)
r.status_code

200

In [342]:
text_response = r.content #https://stackoverflow.com/questions/52389692/beautifulsoup-and-prettify-function
soup = BeautifulSoup(text_response,"html.parser")

In [343]:
table = soup.findAll("table") #find the table in the HTML

In [344]:
solar_flare_df = pd.read_html(str(table))

In [345]:
solfla_df = solar_flare_df[0]#works when we just select the first table

In [346]:
solfla_df.columns = ["rank", "x_classification", "date", "region", "start_time", "maximum_time", "end_time", "movie"] #rename columns to be more descriptive

In [347]:
solfla_df.head()

,rank,x_classification,date,region,start_time,maximum_time,end_time,movie
0,1,X28+,2003/11/04,486,19:29,19:53,20:06,MovieView archive
1,2,X20+,2001/04/02,9393,21:32,21:51,22:03,MovieView archive
2,3,X17.2+,2003/10/28,486,09:51,11:10,11:24,MovieView archive
3,4,X17+,2005/09/07,808,17:17,17:40,18:03,MovieView archive
4,5,X14.4,2001/04/15,9415,13:19,13:50,13:55,MovieView archive


## Step 2 - Tidy the Top 50 Solar Flare Data

In [348]:
solfla_df = solfla_df[["rank", "x_classification", "date", "start_time", "maximum_time", "end_time","region"]]
solfla_df.head()

,rank,x_classification,date,start_time,maximum_time,end_time,region
0,1,X28+,2003/11/04,19:29,19:53,20:06,486
1,2,X20+,2001/04/02,21:32,21:51,22:03,9393
2,3,X17.2+,2003/10/28,09:51,11:10,11:24,486
3,4,X17+,2005/09/07,17:17,17:40,18:03,808
4,5,X14.4,2001/04/15,13:19,13:50,13:55,9415


In [349]:
import datetime

In [350]:
for index, row in solfla_df.iterrows():
    solfla_df["start_time"] = pd.to_datetime(solfla_df["start_time"])
    solfla_df["maximum_time"] = pd.to_datetime(solfla_df["maximum_time"])
    solfla_df["end_time"] = pd.to_datetime(solfla_df["end_time"])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [351]:
solfla_df.drop(columns="date",inplace=True) #drop date table

solfla_df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,rank,x_classification,start_time,maximum_time,end_time,region
0,1,X28+,2021-10-22 19:29:00,2021-10-22 19:53:00,2021-10-22 20:06:00,486
1,2,X20+,2021-10-22 21:32:00,2021-10-22 21:51:00,2021-10-22 22:03:00,9393
2,3,X17.2+,2021-10-22 09:51:00,2021-10-22 11:10:00,2021-10-22 11:24:00,486
3,4,X17+,2021-10-22 17:17:00,2021-10-22 17:40:00,2021-10-22 18:03:00,808
4,5,X14.4,2021-10-22 13:19:00,2021-10-22 13:50:00,2021-10-22 13:55:00,9415


## Step 3 - Scrape the NASA Data

In [352]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
r = requests.get("https://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html",timeout=10,headers=headers)
r.status_code

200

In [353]:
text_response = r.content #https://stackoverflow.com/questions/52389692/beautifulsoup-and-prettify-function
soup = BeautifulSoup(text_response,"html.parser")
table = soup.get_text()
split_table = table.splitlines()
split_table = split_table[15:]
split_table = split_table[0:len(split_table)-2] #clean up table by removing top and bottom portion


In [354]:
nasa_df = pd.DataFrame(
    columns = ["start_date","start_time","end_date","end_time","start_freq","end_freq","location","region","xray_importance","cme_date","cme_time","central_pos_angle","cme_width","cme_speed","PHTX" ]
    )
for i,v in enumerate(split_table):
  row_vals = re.findall(r"\S+",v)
  nasa_df.loc[i] = row_vals[0:15]

In [355]:
nasa_df.drop(columns=["PHTX"],inplace=True)

In [356]:
nasa_df.loc[0:5]

,start_date,start_time,end_date,end_time,start_freq,end_freq,location,region,xray_importance,cme_date,cme_time,central_pos_angle,cme_width,cme_speed
0,1997/04/01,14:00,04/01,14:15,8000,4000,S25E16,8026,M1.3,04/01,15:18,74,79,312
1,1997/04/07,14:30,04/07,17:30,11000,1000,S28E19,8027,C6.8,04/07,14:27,Halo,360,878
2,1997/05/12,05:15,05/14,16:00,12000,80,N21W08,8038,C1.3,05/12,05:30,Halo,360,464
3,1997/05/21,20:20,05/21,22:00,5000,500,N05W12,8040,M1.3,05/21,21:00,263,165,296
4,1997/09/23,21:53,09/23,22:16,6000,2000,S29E25,8088,C1.4,09/23,22:02,133,155,712
5,1997/11/03,05:15,11/03,12:00,14000,250,S20W13,8100,C8.6,11/03,05:28,240,109,227


## Step 4 - Tidy the Nasa Data

**Replace All empty values with NaNs**

In [357]:
nasa_df.region.replace("-----", np.nan,inplace=True)
nasa_df.xray_importance.replace("----", np.nan,inplace=True)
nasa_df.cme_date.replace("--/--",np.nan, inplace=True)
nasa_df.cme_time.replace("--:--",np.nan,inplace=True)
nasa_df.central_pos_angle.replace("----", np.nan,inplace=True)
nasa_df.cme_width.replace("----", np.nan,inplace=True)
nasa_df.cme_speed.replace("----", np.nan,inplace=True)


**Replace all lower case "Back" values with upper case "BACK" in location column**

In [358]:
nasa_df.location.replace("Back","BACK",inplace=True)

**Replace all DSF values with FILA in region column**

In [359]:
nasa_df.region.replace("DSF","FILE",inplace=True)

**Create column to indicate HALO flare, then turn Halo entries into NaN.**<br/>
Halo = 1 <br/>
Non-Halo = 0

In [360]:
column_vals = []
for index,value in nasa_df["central_pos_angle"].iteritems():
  if value == "Halo":
    column_vals.append(1)
  else:
    column_vals.append(0)
nasa_df["is_halo"] = column_vals

In [361]:
nasa_df.central_pos_angle.replace("Halo", np.nan,inplace=True)

**Indicate which cme_widths are lower bounds, make a new column with the info, then remove the non-numeric characters**

In [362]:
column_vals = []
for index,value in nasa_df["cme_width"].iteritems():
  if pd.isnull(value) == True:
    column_vals.append(0)
  elif ">" in value:
        column_vals.append(1)
  else:
    column_vals.append(0)
nasa_df["lower_bound"] = column_vals

In [363]:
def remove_sign(str):
  if pd.isnull(str) == True:
    return str
  return re.sub(">","",str)

nasa_df["cme_width"] = nasa_df["cme_width"].apply(remove_sign)

**Reformat date columns and drop repetitive data**

In [364]:
# for index, row in nasa_df.iterrows():
#     nasa_df["start_date"] = pd.to_datetime(nasa_df["start_date"],format="%Y/%m/%d")
#     nasa_df["start_time"] = pd.to_timedelta( pd.to_datetime(nasa_df["start_time"],format="%H:%S"))
#     nasa_df["start_time"] = nasa_df["start_date"] + nasa_df["start_time"]

In [365]:
display(nasa_df.loc[20:30])

,start_date,start_time,end_date,end_time,start_freq,end_freq,location,region,xray_importance,cme_date,cme_time,central_pos_angle,cme_width,cme_speed,is_halo,lower_bound
20,1998/05/09,03:35,05/09,10:00,9000,400,S14W89,8210,M7.7,05/09,03:35,262,178,2331,0,0
21,1998/05/11,21:40,05/11,22:00,10000,1000,N32W90,8214,B6.6,05/11,21:55,208,301,830,0,1
22,1998/05/19,10:00,05/19,11:30,14000,3000,N23W43,8222,B5.7,05/19,10:27,268,139,801,0,0
23,1998/05/27,13:30,05/27,14:20,4000,1000,N19W62,8226,C7.5,05/27,13:45,175,268,878,0,0
24,1998/06/11,10:15,06/11,10:20,8000,4000,N16E86,8243,M1.4,06/11,10:28,123,177,1223,0,0
25,1998/06/16,18:20,06/17,21:00,12000,50,S22W90,8232,M1.0,06/16,18:27,341,281,1484,0,0
26,1998/06/20,19:39,06/20,20:00,2600,1800,BACK,NaN,NaN,06/20,18:20,NaN,360,964,1,0
27,1998/06/22,07:15,06/22,09:20,6000,2000,N16W46,8243,C2.9,06/22,05:01,265,59,206,0,0
28,1998/11/02,14:00,11/02,14:40,14000,4000,S25E44,8373,C4.4,11/02,14:18,116,169,661,0,0
29,1998/11/05,22:00,11/07,08:00,5000,50,N22W18,8375,M8.4,11/05,20:44,NaN,360,1118,1,0
